In [1]:
import torch
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

# Verificar GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Treinando no dispositivo: {device}")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Carregar o dataset
train_dataset = datasets.ImageFolder(root='/home/abel/Downloads/TCC-PUC-Melanoma-Detection/dataset/train', transform=transform)
val_dataset = datasets.ImageFolder(root='/home/abel/Downloads/TCC-PUC-Melanoma-Detection/dataset/val', transform=transform)

# Configurar os DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

Treinando no dispositivo: cuda


Configuração do Modelo

In [2]:
# Carregar o modelo DenseNet-121 pré-treinado
model_densenet = models.densenet121(pretrained=True)

# Ajustar para 2 classes
num_features = model_densenet.classifier.in_features
model_densenet.classifier = torch.nn.Linear(num_features, 2)

model_densenet = model_densenet.to(device)

/home/abel/anaconda3/envs/pytorch_env/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/abel/anaconda3/envs/pytorch_env/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Configuração do Otimizador e Função de Perda

In [3]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_densenet.parameters(), lr=0.001)

Treino

In [4]:
num_epochs = 5000
for epoch in range(num_epochs):
    model_densenet.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model_densenet(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Época {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")
    model_densenet.cpu()
    
    # Validação
    model_densenet.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.cpu(), labels.cpu()  # Mover os dados para a CPU também
            outputs = model_densenet(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_loss = val_loss / len(val_loader.dataset)
    val_accuracy = correct / total
    print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
    model_densenet.cuda()

Época 1/5000, Loss: 0.2055
Val Loss: 0.2575, Val Accuracy: 0.9345
Época 2/5000, Loss: 0.1896
Val Loss: 0.2094, Val Accuracy: 0.9418
Época 3/5000, Loss: 0.1651
Val Loss: 0.3933, Val Accuracy: 0.8436
Época 4/5000, Loss: 0.1630
Val Loss: 0.3070, Val Accuracy: 0.9273
Época 5/5000, Loss: 0.1419
Val Loss: 0.1866, Val Accuracy: 0.9418
Época 6/5000, Loss: 0.1309
Val Loss: 0.1862, Val Accuracy: 0.9418
Época 7/5000, Loss: 0.1376
Val Loss: 0.1999, Val Accuracy: 0.9455
Época 8/5000, Loss: 0.1314
Val Loss: 0.2746, Val Accuracy: 0.8982
Época 9/5000, Loss: 0.1128
Val Loss: 0.1977, Val Accuracy: 0.9382
Época 10/5000, Loss: 0.1023
Val Loss: 0.2269, Val Accuracy: 0.9382
Época 11/5000, Loss: 0.1033
Val Loss: 0.2333, Val Accuracy: 0.9273
Época 12/5000, Loss: 0.1092
Val Loss: 0.1649, Val Accuracy: 0.9418
Época 13/5000, Loss: 0.0981
Val Loss: 0.1670, Val Accuracy: 0.9527
Época 14/5000, Loss: 0.0909
Val Loss: 0.1788, Val Accuracy: 0.9418
Época 15/5000, Loss: 0.1001
Val Loss: 0.1913, Val Accuracy: 0.9455
Époc

Salvar o modelo

In [5]:
torch.save(model_densenet.state_dict(), '/home/abel/Downloads/TCC-PUC-Melanoma-Detection/result_models/densenet121_melanoma.pth')